In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData,func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,Integer,String,Numeric,Text,Float,ForeignKey
from sqlalchemy.orm import sessionmaker,relationship
import datetime
import dateutil.relativedelta
import quandl
import requests
import time
from sqlalchemy.ext.automap import automap_base
import plotly as py
py.tools.set_credentials_file(username='jiji_v_jacob', api_key="iBN6typcxs3KOaRus216")
import plotly.plotly as py
import plotly.graph_objs as go
from sqlalchemy.orm import Session
import math

In [2]:
engine = create_engine("sqlite:///pub_atx.sqlite")

In [3]:
Base = declarative_base()

In [4]:
# class Company(Base):
#     __tablename__ = 'cmpny'
    
#     id_cmpny = Column(Integer, primary_key=True)
#     rank = Column(Integer)
#     name = Column(Text)
#     address = Column(Text)
#     city = Column(Text)
#     state = Column(Text)
#     zip_code = Column(Text)
#     phn_nbr = Column(Text)
#     website = Column(Text)
#     tckr = Column(Text)
#     ipo_yr = Column(Integer)
#     yr_founded = Column(Integer)
#     gen_buss_desc = Column(Text)
#     curr_top_exec = Column(Text)
#     yr_estblsh = Column(Integer)
#     austin_staff_cnt = Column(Float)
#     comp_staff_cnt = Column(Float)
#     tot_local_emp_cnt = Column(Float)
#     exchng = Column(Text)
#     lat = Column(Float)
#     lng = Column(Float)
#     industry = Column(Text)
#     sector = Column(Text)
#     marketcap = Column(Float)
    
#     def __repr__(self):
#         return f"id={self.id},name=[self.name]"

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
company = Base.classes.cmpny

In [7]:
session = Session(engine)

In [8]:
sectors = session.query(func.distinct(company.sector)).all()

In [9]:
sectors = session.query(company.sector,company.name,company.yr_estblsh,company.marketcap,company.austin_staff_cnt,company.comp_staff_cnt).order_by(company.sector).statement

In [10]:
df_sector_comp = pd.read_sql(sectors,session.bind)

In [11]:
slope = 2.666051223553066e-05
hover_text = []
bubble_size = []
traces = []

In [12]:
df_sector_comp["austin_staff_cnt"] = df_sector_comp["austin_staff_cnt"].astype(int)
df_sector_comp["comp_staff_cnt"] = df_sector_comp["comp_staff_cnt"].astype(int)

In [13]:
df_sector_comp.head()

,sector,name,yr_estblsh,marketcap,austin_staff_cnt,comp_staff_cnt
0,Consumer Cyclical,Chuy's Holdings Inc.,1982,4.515957e+08,893,8200
1,Energy,Parsley Energy LLC,2008,8.161527e+09,0,298
2,Energy,USA Compression Partners LLC,1998,1.098975e+09,26,433
3,Energy,Jones Energy Inc.,1993,9.513039e+07,52,90
4,Energy,FieldPoint Petroleum Corp.,1989,2.133952e+06,3,3


In [14]:
for index, row in df_sector_comp.iterrows():
    hover_text.append(('Name: {name}<br>'+
                      'Market Cap: {marketcap}<br>'+
                      'Year Est. : {yr_estblsh}<br>'+
                      'Staff Count: {comp_staff_cnt}<br>'+
                      'Austin Staff Count: {austin_staff_cnt}').format(name=row['name'],
                                            marketcap=row['marketcap'],
                                            yr_estblsh=row['yr_estblsh'],
                                            comp_staff_cnt=row['comp_staff_cnt'],
                                            austin_staff_cnt=row['austin_staff_cnt']))
    bubble_size.append(math.sqrt(row['comp_staff_cnt']*slope))

In [15]:
df_sector_comp['text'] = hover_text
df_sector_comp['size'] = bubble_size
sizeref = 2.*max(df_sector_comp['size'])/(100**2)

In [16]:
sectors = df_sector_comp["sector"].unique()

In [17]:
sectors

array(['Consumer Cyclical', 'Energy', 'Financial Services', 'Healthcare',
       'Industrials', 'Real Estate', 'Technology'], dtype=object)

In [18]:
for sector in sectors:
    trace = go.Scatter(
                        x=df_sector_comp['yr_estblsh'][df_sector_comp['sector'] == sector],
                        y=df_sector_comp['marketcap'][df_sector_comp['sector'] == sector],
                        mode='markers',
                        name= sector,
                        text=df_sector_comp['text'][df_sector_comp['sector'] == sector],
                        marker=dict(
                                    sizemode='area',
                                    sizeref=sizeref,
                                    size=df_sector_comp['size'][df_sector_comp['sector'] == sector],
                                    line=dict(
                                    width=2
                                    ),
                                   )
                      )
    traces.append(trace)

In [19]:
print(traces)

[{'type': 'scatter', 'x': 0    1982
Name: yr_estblsh, dtype: int64, 'y': 0    451595710.0
Name: marketcap, dtype: float64, 'mode': 'markers', 'name': 'Consumer Cyclical', 'text': 0    Name: Chuy's Holdings Inc.<br>Market Cap: 4515...
Name: text, dtype: object, 'marker': {'sizemode': 'area', 'sizeref': 9.3512822721026107e-05, 'size': 0    0.467564
Name: size, dtype: float64, 'line': {'width': 2}}}, {'type': 'scatter', 'x': 1    2008
2    1998
3    1993
4    1989
Name: yr_estblsh, dtype: int64, 'y': 1    8.161527e+09
2    1.098975e+09
3    9.513039e+07
4    2.133952e+06
Name: marketcap, dtype: float64, 'mode': 'markers', 'name': 'Energy', 'text': 1    Name: Parsley Energy LLC<br>Market Cap: 816152...
2    Name: USA Compression Partners LLC<br>Market C...
3    Name: Jones Energy Inc.<br>Market Cap: 9513039...
4    Name: FieldPoint Petroleum Corp.<br>Market Cap...
Name: text, dtype: object, 'marker': {'sizemode': 'area', 'sizeref': 9.3512822721026107e-05, 'size': 1    0.089134
2    0.10744

In [20]:
layout = go.Layout(
    title='Year Established v. Market Cap',
    xaxis=dict(
        title='Year Established',
        gridcolor='rgb(255, 255, 255)',
        range=[df_sector_comp["yr_estblsh"].min() - 10 , df_sector_comp["yr_estblsh"].max() + 10],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Market Cap',
        gridcolor='rgb(255, 255, 255)',
        range=[df_sector_comp["marketcap"].min() - 1000000000, df_sector_comp["marketcap"].max() + 1000000000],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

In [21]:
fig = go.Figure(data=traces, layout=layout)


In [22]:
py.iplot(fig, filename='atxPublicCompanies')